In [1]:
import requests
from fastapi import FastAPI, File, UploadFile
import cv2
import numpy as np

In [2]:
def decode_qr_online(image_path):
    with open(image_path, 'rb') as image_file:
        print()
        response = requests.post(
            'https://api.qrserver.com/v1/read-qr-code/',
            files={'file': image_file}
        )
    decoded_info = response.json()
    if decoded_info:
        print('Decoded value is:', decoded_info[0]['symbol'][0]['data'])
    else:
        print('No QR code found')

In [3]:
decode_qr_online("../data/vpay_test/202309271344459586.jpg")


Decoded value is: 0041000600000101030040220013270134700BPM093485102TH9104F55C


In [4]:
import cv2
import numpy as np

def decode_qr_with_opencv(image_path):
    # 1. Load the image using OpenCV
    image = cv2.imread(image_path)
    
    # Check if the image is loaded successfully
    if image is None:
        print("Error: Couldn't load the image.")
        return

    # 2. Initialize the QRCodeDetector
    qr_code_detector = cv2.QRCodeDetector()

    # 3. Decode the QR code
    retval, decoded_info, _ = qr_code_detector.detectAndDecode(image)

    # 4. Check the decoded info and print the result
    if retval:
        print('Decoded value is:', retval)
    else:
        print('No QR code found')

In [5]:
decode_qr_with_opencv('../data/vpay_test/202309271344459586.jpg')

Decoded value is: 0041000600000101030040220013270134700BPM093485102TH9104F55C


In [6]:
import requests
from PIL import Image

MAX_RETRIES = 3

def send_image_for_qr_decoding(image_path):
    # File validation
    try:
        with Image.open(image_path) as img:
            if img.format not in ['JPEG', 'PNG']:
                print("Unsupported image format.")
                return
    except Exception as e:
        print(f"Error in reading the image: {e}")
        return

    retry_count = 0
    while retry_count < MAX_RETRIES:
        try:
            with open(image_path, 'rb') as image_file:
                response = requests.post(
                    'http://127.0.0.1:8000/decode-qr/',
                    files={'file': image_file}
                )
            
            decoded_info = response.json()
            if decoded_info and 'decoded_value' in decoded_info:
                print('Decoded value is:', decoded_info['decoded_value'])
                break
            else:
                print('No QR code found or error in decoding.')
                break
        except Exception as e:
            print(f"Error in request: {e}")
            retry_count += 1
            if retry_count < MAX_RETRIES:
                print(f"Retrying... ({retry_count}/{MAX_RETRIES})")
            else:
                print("Max retries reached.")



In [7]:
send_image_for_qr_decoding('../data/202309272014363302.jpg')

Error in reading the image: [Errno 2] No such file or directory: '../data/202309272014363302.jpg'


In [13]:
def word_romanized_check(to_check_name, real_eng_name, server_url="http://127.0.0.1:8000"):
    url = f"{server_url}/name-checker/"

    data = {
        "to_check_name": to_check_name,
        "true_name": real_eng_name
    }

    try:
        response = requests.post(url, data=data)

        if response.status_code == 200:
            return response.json()
        else:
            print(f"Error (response.status_code): {response.text}")
            return None
        
    except Exception as e:
        print(f"Error in sending quest: {e}")
        return None

In [14]:
word_romanized_check("ปฐวี", "patawee")

{'to_check_th_romanized': 'pathawi',
 'is_match': True,
 'match_score': 71.42857142857143}

In [16]:
# Example usage
patawee_result = word_romanized_check("Panrawee", "Panrawee")
vipoo_result = word_romanized_check("วิภู", "Vipoo")
panrawee_result = word_romanized_check("Panrawee", "Panrawee")
print(patawee_result)
print(vipoo_result)
print(panrawee_result)

{'is_match': True, 'match_score': 100.0}
{'to_check_th_romanized': 'wiphu', 'is_match': False, 'match_score': 40.0}
{'is_match': True, 'match_score': 100.0}


In [11]:
from lingua import Language, LanguageDetectorBuilder

languages = [Language.ENGLISH, Language.THAI]
detector = LanguageDetectorBuilder.from_languages(*languages)\
.with_minimum_relative_distance(0.9)\
.build()

In [12]:
print(detector.detect_language_of("lfdดีจ้าe is english"))

Language.THAI
